# Strong Maximum Test design point validation

In this example we are going to validate a FORM estimation using the Strong Maximum Test.

The Strong Maximum Test helps to evaluate the quality of the design point
resulting from the optimization algorithm. It checks whether the design point computed is:

- the **true** design point, which means a global maximum point,
- a **strong** design point, which means that there is no other local maximum
  located on the event boundary and which likelihood is slightly inferior to the design point one.

This verification is very important in order to give sense to the FORM and SORM approximations.

We briefly recall here the main principles of the test.
The objective is to detect all the points $\tilde{P}^*$ in the ball of radius
$R_{\varepsilon} = \beta(1+\delta_{\varepsilon})$ which are potentially the
real design point (case of $\tilde{P}_2^*$) or which contribution to $P_f$ is
not negligeable as regards the approximations Form and Sorm (case of $\tilde{P}_1^*$).
The contribution of a point is considered as negligeable when its likelihood in
the $U$-space is more than $\varepsilon$-times lesser than the design point one.
The radius $R_{\varepsilon}$ is the distance to the $U$-space center upon which
points are considered as negligeable in the evaluation of $P_f$.


In [2]:
from __future__ import print_function
import openturns as ot

In [3]:
# Create the model Y = x1 + 2*x2
model = ot.SymbolicFunction(["x1", "x2"], ["x1+2*x2"])

# Create the input distribution and random vector X
inputDist = ot.Normal(2)
inputDist.setDescription(['X1', 'X2'])

inputVector = ot.RandomVector(inputDist)

# Create the output random vector Y=model(X)
output = ot.CompositeRandomVector(model, inputVector)
output.setName("MyOutputY")

In [4]:
# Create the physical event Y > 4
threshold = 4
myEvent = ot.Event(output, ot.Greater(), threshold)

# Create the associated standard event in the standard space
myStandardEvent = ot.StandardEvent(myEvent)

In [5]:
# First : FORM analyses to get the design point
myCobyla = ot.Cobyla()
myStartingPoint = inputDist.getMean()
myAlgoFORM = ot.FORM(myCobyla, myEvent, myStartingPoint)
myAlgoFORM.run()
FORMResult = myAlgoFORM.getResult()
standardSpaceDesignPoint = FORMResult.getStandardSpaceDesignPoint()

In [6]:
# Fix the importance level epsilon of the test
# Care : 0<epsilon<1
importanceLevel = 0.15

# Fix the accuracy level tau of the test
# Care : tau >0
# It is recommended that tau <4
accuracyLevel = 3.0

# Fix the confidence level (1-q) of the test
confidenceLevel = 0.99

# Create the Strong Maximum Test
# CARE : the event must be declared in the standard space
# 1. From the confidenceLevel parameter
mySMT_CL = ot.StrongMaximumTest(myStandardEvent, standardSpaceDesignPoint,  importanceLevel, accuracyLevel,  confidenceLevel)

# 2. Or from the  maximum number of points sampling the sphere
pointsNumber = 1000
mySMT_PN = ot.StrongMaximumTest(myStandardEvent, standardSpaceDesignPoint,  importanceLevel, accuracyLevel,  pointsNumber)

# Perform the test
mySMT_CL.run()
mySMT_PN.run()

# Get (or evaluate) the confidence level
# associated to the number of points used to sample the sphere
print('Confidence level = ', mySMT_CL.getConfidenceLevel())

# Get (or evaluate) the number of points used to sample the sphere
# associated the confidence level used
print('Points Number = ', mySMT_CL.getPointNumber())

Confidence level =  0.99
Points Number =  13


In [7]:

# Get all the points verifying the event  and outside the design point vicinity
# Get also the values of limit state function at these points
potentialDesignPoints = mySMT_CL.getFarDesignPointVerifyingEventPoints()
values = mySMT_CL.getFarDesignPointVerifyingEventValues()
print('Potential design points = ', potentialDesignPoints)
print('Model values = ', values)

# Get all the points verifying the event and inside the design point vicinity
# Get also the values of limit state function at these points
vicinityDesignPoint = mySMT_CL.getNearDesignPointVerifyingEventPoints()
values = mySMT_CL.getNearDesignPointVerifyingEventValues()
print('Points verifying the Event in the vicinity of the design points = ', vicinityDesignPoint)
print('Model values = ', values)

# Get all the points not verifying the event and outside the design point vicinity
# Get also the values of limit state function at these points
farSecurityPoints = mySMT_CL.getFarDesignPointViolatingEventPoints()
values = mySMT_CL.getFarDesignPointViolatingEventValues()
print('Points NOT verifying the Event outside the vicinity of the design points = ', farSecurityPoints)
print('Model values = ', values)

# Get  all the points not verifying the event and inside the design point vicinity
# Get also the values of limit state function at these points
vicinitySecurityPoints = mySMT_CL.getNearDesignPointViolatingEventPoints()
values = mySMT_CL.getNearDesignPointViolatingEventValues()
print('Points NOT verifying the Event outside the vicinity of the design points = ', vicinitySecurityPoints)
print('Model values = ', values)

Potential design points =  
Model values =  
Points verifying the Event in the vicinity of the design points =  0 : [  2.5556   3.5279  ]
1 : [ -1.48846  4.0941  ]
2 : [  3.1138   3.04654 ]
3 : [ -1.04463  4.22918 ]
Model values =      [ y0      ]
0 : [ 9.6114  ]
1 : [ 6.69974 ]
2 : [ 9.20688 ]
3 : [ 7.41373 ]
Points NOT verifying the Event outside the vicinity of the design points =  0 : [ -3.8094    2.11321  ]
1 : [ -3.80056  -2.12906  ]
2 : [ -4.34588  -0.300764 ]
3 : [  4.31418  -0.604166 ]
4 : [ -3.40961   2.71141  ]
5 : [ -4.30125   0.690212 ]
6 : [ -3.35984  -2.77284  ]
7 : [  4.21095  -1.11581  ]
8 : [  1.8862   -3.92675  ]
Model values =      [ y0        ]
0 : [  0.417029 ]
1 : [ -8.05868  ]
2 : [ -4.94741  ]
3 : [  3.10585  ]
4 : [  2.01321  ]
5 : [ -2.92083  ]
6 : [ -8.90552  ]
7 : [  1.97934  ]
8 : [ -5.96731  ]
Points NOT verifying the Event outside the vicinity of the design points =  
Model values =  
